In [25]:
import os, subprocess
import xml.sax
import mwparserfromhell 
import json
import re
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import sys
import os
from multiprocessing import Pool 
import tqdm 
from itertools import chain
from functools import partial
import gc
import json

In [36]:
dumps = os.listdir('wiki_dump/')
os.stat('wiki_dump/' + dumps[14]).st_size

4096

In [37]:
data_path = 'wiki_dump/' + dumps[0]

In [38]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._articles = []
        self._total_articles = 0
        self._articles_count = 0

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._total_articles += 1
            # Search through the page to see if the page is a target_article
            article = process_article(**self._values)
            # Append to the list of articles
            if article:
                self._articles_count += 1
                self._articles.append(article)

In [39]:
def process_article(title, text, timestamp):
    """Search wiki_dump for desired articles"""
    
    wiki = mwparserfromhell.parse(text)
    
    if title in input_titles:

        striped_code = wiki.strip_code().strip()
        

        headings = wiki.filter_headings()


        wikilinks = []
        for i in wiki.filter_wikilinks():
            i = i.strip('[]')
            if 'File:' not in i and \
            'Category:' not in i and \
            'Wikipedia:' not in i and \
            'en:' not in i:
                wikilinks.append(i)

        return (title, striped_code, headings, wikilinks)

In [ ]:
start = timer()

handler = WikiXmlHandler()

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    if (i + 1) % 10000 == 0:
        sys.stdout.write('\r' +f'Processed {i + 1} lines so far.')
    try:
        parser.feed(line)
    except StopIteration:
        break
    
end = timer()

print(f'\nSearched through {handler._articles_count} articles.')
print(f'\nFound {len(articles)} articles in {round(end - start)} seconds.')

Processed 12540000 lines so far.

In [ ]:
articles = handler._articles

In [ ]:
handler._total_articles

In [ ]:
articles[0]